In [317]:
import requests
import pandas as pd
import datetime
import sqlite3
import json

In [304]:
from statistics import mean, median, mode

In [305]:
# OFAC publishes a list of Sectoral Sanctions Identifications (SSI), 
# which lists persons, companies, and entities in sectors of the Russian economy 
# (especially energy, finance, and armaments), prohibiting certain types of activity with these 
# individuals or entities by United States persons, wherever located.

In [306]:
r = requests.get('https://www.treasury.gov/ofac/downloads/ssi/ssilist.txt')

In [307]:
lines = str(r.text).split("\n")

In [308]:
f_list = []
entry_list = []

In [309]:
for line in lines:
    if line != '':
        entry_list.append(line)        
    else:
        f_list.append(entry_list)
        entry_list = []

In [310]:
count = 1
ff_list = []

In [311]:
while count < len(f_list):
    if len(f_list[count]) == 1:
        if count == len(f_list) - 2:
            break
        count += 2
    ff_list.append(f_list[count])
    count += 1

In [312]:
fff_list = []
for lines in ff_list:
    s = str()
    for line in lines:
        if 'http://' in line:
            s += line
        elif 'https://' in line:
            s += line
        elif 'center/' in line:
            s += line
        else: 
            s += (line + ' ')
    fff_list.append(s)

In [392]:
ffff_list = []
ffff_list_other = []
for entry in fff_list:
    l = []
    entry = entry.split(';')
    if '(' in entry[0]:
        # if the 0 item of the list has any a.k.a.'s at all
        if ')' not in entry[0]:
            # if the 0 item of the list has only an open parenthesis and no close
            # then follow this logic, which will assume a.k.a.'s exist on multiple items
            a = entry[0].split('(')
            # split from the '(' open parenthisis to create [0] item for name, and [1] item for first a.k.a.
            l.append(a[0].rstrip())
            l.append(a[1].rstrip())
        count = 1
        count_copy = 1 # will use this to keep track of where we are within the entry later on
        while count < len(entry):
            if 'a.k.a.' in entry[count]:
                if ')' not in entry[count]:
                    # check to see if the line is not the final a.k.a.
                    l.append(entry[count][1:])
                elif '),' in entry[count]:
                    # check to see if the line is the final a.k.a.
                    # if so, split on the close parenthesis to create [0] for final a.k.a., and [1] for address
                    a = entry[count].split('),')
                    l.append(a[0][1:])
                    l.append(a[1][1:])
                    count_copy = count + 1 # using count_copy to ensure we can append the rest of the info from entry
                    break
            else:
                count_copy = count + 1
                break
            count += 1
        while count_copy < len(entry):
            # the rest of the lines are straightforward, and we will just append them
            l.append(entry[count_copy][1:].rstrip())
            count_copy += 1
        ffff_list.append(l)
    else:
        l = []
        count = 0
        while count < len(entry):
            if count == 0:
                l.append(entry[count])
            else:
                l.append(entry[count][1:].rstrip())
            count += 1
        ffff_list_other.append(l)

In [393]:
ffff_list_other[1]

['BOM FINANCE LTD, 2nd Floor Vanterpool Plaza, Wickhams Cay 1, Road Town, Virgin Islands, British',
 'Executive Order 13662 Directive Determination - Subject to Directive 1',
 'For more information on directives, please visit the following link: https://www.treasury.gov/resource-center/sanctions/Programs/Pages/ukraine.aspx#directives [UKRAINE-EO13662] (Linked To: BANK OF MOSCOW).']

In [394]:
ffff_list[1]

['AGROKREDIT-INFORM, AO',
 "a.k.a. AKTSIONERNOE OBSHCHESTVO 'AGROKREDIT- INFORM'",
 "a.k.a. CLOSED JOINT-STOCK COMPANY 'AGROCREDIT-INFORM'",
 '3 per. Gagarinski, Moscow 119034, Russia',
 '3 Gagarinsky Pereulok, Moscow, Russia',
 'Executive Order 13662 Directive Determination - Subject to Directive 1',
 'Registration ID 1087746334400',
 'Tax ID No. 7704681172',
 'Government Gazette Number 85651516',
 'For more information on directives, please visit the following link: http://www.treasury.gov/resource-center/sanctions/Programs/Pages/ukraine.aspx#directives. [UKRAINE-EO13662] (Linked To: RUSSIAN AGRICULTURAL BANK).']